In [2]:
import pandas as pd

In [52]:

log_data  = open('review_screp.out', 'r')

    
execution = []
count = []
mem_usage = []

for line in log_data:
    elements = line.split('::')
    datetime = elements[0].split(' ')
    date = datetime[0]
    time = datetime[1]
    
    if 'memory used in MB' in elements[1]:
        mem_used_in_mb = elements[1].split(':')[1].strip().split(' ')[0]
        mem_usage.append([date, time, mem_used_in_mb])
    elif 'Reviews scanned so far' in elements[1]:
        reviews_scanned = elements[1].split('&&')[0].split(':')[1].strip()
        count.append([date, time, reviews_scanned])
    elif 'date in process:' in elements[1]:
        split_log = elements[1].split(',')
        extract_from = split_log[0].split(' ')
        day = extract_from[-1]
        month = extract_from[-2]
        year = split_log[1].strip()
        ignored = 'is before our start date!!' in split_log[-1]
            
        execution.append([date, time, day, month, year, ignored])

                   
df_execution = pd.DataFrame(execution, columns=['date', 'time', 'review_date', 'review_month', 'review_year', 'is_ignored'])
df_count = pd.DataFrame(count, columns=['date', 'time', 'reviews_scanned'])
df_mem_usage = pd.DataFrame(mem_usage, columns=['date', 'time', 'mem_used_in_MB'])
df_execution.tail()

,date,time,review_date,review_month,review_year,is_ignored
7098,30/10/2021,10:26:20,2,January,2018,False
7099,30/10/2021,10:26:21,2,January,2018,False
7100,30/10/2021,11:04:37,2,January,2018,False
7101,30/10/2021,12:21:00,1,January,2018,False
7102,30/10/2021,12:21:02,1,January,2018,False
